In [43]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Stop warnings
# -------------
import warnings
warnings.filterwarnings("ignore")

# General imports
# ---------------
import os
import sys
import json
import glob
import numpy as np
import matplotlib.pyplot as pl
import ipdb
import platform
opj = os.path.join
deb = ipdb.set_trace

# MRI imports
# -----------
import nibabel as nb
import cortex
from nipype.interfaces.freesurfer import SurfaceTransform

subject = 'sub-02'

# Define analysis parameters
# --------------------------
with open('projects/retino_HCP/settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)

# Define cluster/server specific parameters
# -----------------------------------------
if 'aeneas' in platform.uname()[1]:
    base_dir = analysis_info['aeneas_base_folder'] 
elif 'lisa' in platform.uname()[1]:
    base_dir = analysis_info['lisa_base_folder'] 

# convert time series to fsaverage per hemisphere
for hemi in ['L','R']:
    data = []
    data_file  =  sorted(glob.glob(opj(base_dir,'raw_data',subject,"{sub}_task-prf_space-fsaverage6_hemi-{hemi}.func_sg_psc.gii".format(hemi = hemi, sub = subject))))

# Resample gii to fsaverage
# -------------------------
print('converting fsavereage6 files to fsaverage')
sxfm = SurfaceTransform()
sxfm.inputs.source_subject = "fsaverage6"
sxfm.inputs.target_subject = "fsaverage"
sxfm.terminal_output = 'none'

for hemi in ['L','R']:
    sxfm.inputs.subjects_dir = opj(base_dir,'derivatives','freesurfer')
    if hemi == 'L': sxfm.inputs.hemi = "lh"
    elif hemi == 'R': sxfm.inputs.hemi = "rh"

    sxfm.inputs.source_file = opj(base_dir,'raw_data',subject,"{sub}_task-prf_space-fsaverage6_hemi-{hemi}.func_sg_psc.gii".format(hemi = hemi, sub = subject))
    sxfm.inputs.out_file = opj(base_dir,'raw_data',subject,"{sub}_task-prf_space-fsaverage_hemi-{hemi}.func_sg_psc.gii".format(hemi = hemi, sub = subject))
    print(sxfm.inputs.out_file)
    sxfm.run()    


converting fsavereage6 files to fsaverage
/home/shared/2018/visual/nprf_indiv/raw_data/sub-02/sub-02_task-prf_space-fsaverage_hemi-L.func_sg_psc.gii
/home/shared/2018/visual/nprf_indiv/raw_data/sub-02/sub-02_task-prf_space-fsaverage_hemi-R.func_sg_psc.gii


In [44]:
# Combine hemispheres
tc_mat=[]
for hemi in ['L','R']:
    tc_out = opj(base_dir,'raw_data',subject,"{sub}_task-prf_space-fsaverage_hemi-{hemi}.func_sg_psc.gii".format(hemi = hemi, sub = subject))
    tc_file = nb.load(tc_out)
    exec('tc_mat_{hemi} = (np.array([tc_file.darrays[i].data for i in range(len(tc_file.darrays))]))'.format(hemi = hemi))
    
tc_mat = np.hstack((tc_mat_L,tc_mat_R))

# save combined file
darrays = [nb.gifti.gifti.GiftiDataArray(d) for d in tc_mat]
gii_out = nb.gifti.gifti.GiftiImage(header = data_file_load.header, extra = data_file_load.extra, darrays = darrays)
nb.save(gii_out,opj(base_dir,'raw_data',subject,"{sub}_task-prf_space-fsaverage.func_sg_psc.gii".format(hemi = hemi, sub = subject)))

In [45]:
# load LR 
file = opj(base_dir,'raw_data',subject,"{sub}_task-prf_space-fsaverage.func_sg_psc.gii".format(hemi = hemi, sub = subject))
tc_file = nb.load(file)
tc_mat = (np.array([tc_file.darrays[i].data for i in range(len(tc_file.darrays))]))

In [46]:
vert = cortex.Vertex(data = tc_mat, subject='fsaverage', vmin=-2.0, vmax=2.0,with_curvature = True,with_labels = True, alpha = 0.5,curvature_brightness = 0.95,curvature_contrast = 0.05)
ds = cortex.Dataset(time_course = vert)
cortex.webgl.show(ds, with_labels=True, with_rois=True,with_label = True,with_curvature = True,curvature_brightness = 0.95,curvature_contrast = 0.05,port=33334)

Started server on port 33334


<JS: window.viewer>